In [2]:
library('dataRetrieval')
library('tidyverse')

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   2.1.1     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [ ]:
#Determine which stations in Alaska have sub-daily temperature data available
gages <- readNWISdata(stateCd="AK", parameterCd="00060", service="uv") %>%
  distinct(site_no)

#Using that created list to pull all the temperature data from USGS server
temperature_gages = vector()
for(i in 1:nrow(gages)){
  temp <- readNWISuv(gages[i,], parameterCd = '00060')
  temperature_gages <- rbind(temperature_gages,temp)
}

temperature_gages = temperature_gages %>%
  mutate(site_no = as.numeric(site_no)) %>%
  mutate(dateTime = as.POSIXct(dateTime, format = "%Y-%m-%d %H:%M:%S")) %>%
  mutate(Year = year(dateTime), Month = month(dateTime), Day = day(dateTime), 
         Hour = hour(dateTime), Minute = minute(dateTime), Second = second(dateTime)) %>%
  mutate(wt_year = ifelse(as.numeric(Month)>=10, as.numeric(Year) + 1, as.numeric(Year)))

temperature_gages = temperature_gages %>%
  mutate(Hour = ifelse(Hour<3, 3, ifelse(Hour < 6, 6, ifelse(Hour<9, 9, ifelse(Hour<12, 12, ifelse(Hour<15, 15, ifelse(Hour<18, 18, ifelse(Hour<21, 21, ifelse(Hour<24,24))))))))) %>%
  group_by(site_no, Year, Month, Day, Hour) %>%
  summarise(meanQ = mean(X_00060_00000)) 

temperature_gages = temperature_gages %>%
  filter(Year < 2021)

write.csv(temperature_gages, file = "/glade/u/home/dblaskey/RBM/Validation/flow_sites_3hr.csv")